In [4]:
# start_date = '2016-01-01'
# end_date = '2021-06-01'
target = "10d_ret_standscalar"
alpha_name = "AlphaNetV1"
shift = 2
sequence = 15
LR = [0.001,0.00001,0.0000001]
epoch_num = [30,30,30]

In [5]:
alpha_list = [
'open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover',
#             'open_ts_norm_30d',
#              'high_ts_norm_30d',
#              'low_ts_norm_30d',
#              'close_ts_norm_30d',
#              'vwap_ts_norm_30d',
#              'volume_ts_norm_30d',
#              'pct_chg_ts_norm_30d',
#              'turnover_ts_norm_30d',
#              'free_turnover_ts_norm_30d'
             ]
feat_list = alpha_list.copy()
method = ["COV","CORR","STD","ZSCORE","RETURN","DECAY"]
from AlphaNet.Data import generate_alpha_list
alpha_list = generate_alpha_list(feat_list,method,day=10)
alpha_list
# alpha_list = [i +"_residual" for i in feat_list]

['COV_open_high_10',
 'COV_open_low_10',
 'COV_open_close_10',
 'COV_open_vwap_10',
 'COV_open_volume_10',
 'COV_open_pct_chg_10',
 'COV_open_turnover_10',
 'COV_open_free_turnover_10',
 'COV_high_low_10',
 'COV_high_close_10',
 'COV_high_vwap_10',
 'COV_high_volume_10',
 'COV_high_pct_chg_10',
 'COV_high_turnover_10',
 'COV_high_free_turnover_10',
 'COV_low_close_10',
 'COV_low_vwap_10',
 'COV_low_volume_10',
 'COV_low_pct_chg_10',
 'COV_low_turnover_10',
 'COV_low_free_turnover_10',
 'COV_close_vwap_10',
 'COV_close_volume_10',
 'COV_close_pct_chg_10',
 'COV_close_turnover_10',
 'COV_close_free_turnover_10',
 'COV_vwap_volume_10',
 'COV_vwap_pct_chg_10',
 'COV_vwap_turnover_10',
 'COV_vwap_free_turnover_10',
 'COV_volume_pct_chg_10',
 'COV_volume_turnover_10',
 'COV_volume_free_turnover_10',
 'COV_pct_chg_turnover_10',
 'COV_pct_chg_free_turnover_10',
 'COV_turnover_free_turnover_10',
 'CORR_open_high_10',
 'CORR_open_low_10',
 'CORR_open_close_10',
 'CORR_open_vwap_10',
 'CORR_open_

In [3]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import concat_original_data , generate_shift_data

In [7]:
concat_original_data(alpha_name=alpha_name,alpha_list=alpha_list)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template
['COV_open_high_5', 'COV_open_high_10', 'COV_open_low_5', 'COV_open_low_10', 'COV_open_close_5', 'COV_open_close_10', 'COV_open_vwap_5', 'COV_open_vwap_10', 'COV_open_volume_5', 'COV_open_volume_10', 'COV_open_pct_chg_5', 'COV_open_pct_chg_10', 'COV_open_turnover_5', 'COV_open_turnover_10', 'COV_open_free_turnover_5', 'COV_open_free_turnover_10', 'COV_high_low_5', 'COV_high_low_10', 'COV_high_close_5', 'COV_high_close_10', 'COV_high_vwap_5', 'COV_high_vwap_10', 'COV_high_volume_5', 'COV_high_volume_10', 'COV_high_pct_chg_5', 'COV_high_pct_chg_10', 'COV_high_turnover_5', 'COV_high_turnover_10', 'COV_high_free_turnover_5', 'COV_high_free_turnover_10', 'COV_low_close_5', 'COV_low_close_10', 'COV_low_vwap_5', 'COV_low_vwap_10', 'COV_low_volume_5', 'COV_low_volume_10', 'COV_low_pct_chg_5', 'COV_low_pct_chg_10', 'COV_low_turnover_5', 'COV_low_turnover_10', 'COV_low_free_turnover_5', 'COV_low_free

100%|██████████| 1305/1305 [02:05<00:00, 10.40it/s]


COV_open_high_5  COV_open_low_5  COV_open_close_5  \
timestamp           ticker                                                      
2016-01-19 09:30:00 600651     5.368270e+07    4.702556e+07      4.223370e+07   
                    600652     2.532270e+07    2.607722e+07      2.107440e+07   
                    600653     2.009683e+06    1.937380e+06      7.154004e+05   
                    600601     6.973919e+05    1.058082e+06      2.416237e+05   
                    600602     5.035882e+04    6.402660e+04      2.268610e+04   
...                                     ...             ...               ...   
2021-05-31 09:30:00 600662    -2.814388e+01    8.774227e+01     -1.468828e+01   
                    002271    -4.977529e+01    2.364059e+02     -2.308410e+02   
                    600460    -5.074994e+01    2.622953e+02     -2.846660e+01   
                    300347    -2.305965e+02    4.947830e+02     -5.462716e+02   
                    600653    -1.011282e+03    1.263908e+04     -4.044573e+03   

                            COV_open_vwap_5  COV_open_volume_5  \
timestamp           ticker                                       
2016-01-19 09:30:00 600651     4.820059e+07       2.139030e+09   
                    600652     2.868260e+07       1.432284e+08   
                    600653     1.935945e+06      -1.234271e+07   
                    600601     7.765358e+05       1.502199e+07   
                    600602     6.156502e+04       6.644284e+05   
...                                     ...                ...   
2021-05-31 09:30:00 600662     1.180891e+01      -2.664634e+05   
                    002271    -2.526765e+02      -6.665211e+05   
                    600460     8.058236e+01      -1.958004e+06   
                    300347    -7.649435e+01      -4.760519e+05   
                    600653     5.272953e+03      -4.008720e+06   

                            COV_open_pct_chg_5  COV_open_turnover_5  \
timestamp           ticker                                            
2016-01-19 09:30:00 600651        -7802.425781         27829.572266   
                    600652        -2063.828613          2322.543457   
                    600653        -3005.053955           -70.684517   
                    600601        -2588.298096            68.425301   
                    600602         -723.775208             7.553238   
...                                        ...                  ...   
2021-05-31 09:30:00 600662          -19.721022            -2.529929   
                    002271          -64.982033            -3.830641   
                    600460          -53.585579           -14.923382   
                    300347          -77.322899            -8.387373   
                    600653          -92.978203           -20.594202   

                            COV_open_free_turnover_5  COV_high_low_5  \
timestamp           ticker                                             
2016-01-19 09:30:00 600651              35639.144531    5.466052e+07   
                    600652               2571.508789    2.718039e+07   
                    600653                -82.298264    1.478823e+06   
                    600601                 77.497604    5.650079e+05   
                    600602                 12.622298    3.977791e+04   
...                                              ...             ...   
2021-05-31 09:30:00 600662                 -4.599493   -2.977297e+01   
                    002271                 -3.653948    1.235333e+02   
                    600460                -24.904425    4.038690e+00   
                    300347                 -9.352359    1.256400e+02   
                    600653                -26.719019    2.527927e+03   

                            COV_high_close_5  ...  RETURN_free_turnover_10  \
timestamp           ticker                    ...                            
2016-01-19 09:30:00 600651      6.126268e+07  ...                 4.241930   
                    

# Shift

In [ ]:
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target=target)

100%|██████████| 1705/1705 [00:26<00:00, 65.12it/s] 


[28, 26, 24, 22, 20, 18, 16, 14, 12, 10, 8, 6, 4, 2, 0]


 96%|█████████▌| 1229/1277 [13:54<00:34,  1.39it/s]

# Task Generation

In [ ]:
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
original = task_info["Task"]
display(original)
# Timelist
time_list = ["2019-01-01","2019-06-01","2020-01-01","2020-06-01","2021-01-01","2021-06-01"]
time_list2 = []
for i in range(len(time_list)-1):
    time_list2.append([time_list[i],time_list[i+1]])
time_list2 = pd.DataFrame(time_list2,columns=["start_date","end_date"])
display(time_list2)

In [ ]:
task = pd.DataFrame([
                    alpha_name + "_Shift_%i_Sequence_%i_%s" % (shift,sequence,target),
                    sequence,LR,epoch_num,len(alpha_list),
                        ],index=["Alpha_Name","sequence","LR","epoch_num","feature_num"]).T
task['value']=1
time_list2['value']=1
task = pd.merge(task,time_list2,how='left',on='value')
del task['value']
task["status"] = "waiting"
task["description"] = [{
    "target" : target,
    "alpha_name" : alpha_name,
    "alpha_list" : alpha_list,
    "shift" : shift,
    "sequence" : sequence
} for i in range(len(task))]
task

In [ ]:
task = pd.concat([original,task],axis=0)
task.reset_index(drop=True,inplace=True)
task.index.names = ["task_id"]
task_info["Task"] = task
task[-20:]

In [ ]:
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)